In [ ]:
using LightGraphs
using GraphPlot
using DataStructures
using Colors

In [ ]:
G = DiGraph(6)
E = [
    (1, 2, 16),
    (1, 3, 13),
    (3, 2, 4),
    (2, 4, 12),
    (4, 3, 9),
    (3, 5, 14),
    (4, 6, 20),
    (5, 4, 7),
    (5, 6, 4)
]

C = zeros(Int, nv(G), nv(G))

for e in E
    u, v, c = e
    add_edge!(G, u, v)
    C[u, v] = c
end

function dijkstra_ssp(G, E, W)
    Q = PriorityQueue{Int, Int}()
    D = fill(typemax(Int8), nv(G))
    V = zeros(Bool, nv(G))
    D[1] = 0
    enqueue!(Q, 1, D[1])
    
    
    # returns true if no more steps
    function single_step()
        # IMPLEMENTER DENNE
        if length(Q) > 0
            node = dequeue!(Q)
            while V[node]
                node = dequeue!(Q)
            end
            V[node] = true
            dist = D[node]
            for nb in out_neighbors(G, node)
                if !V[nb] && dist + W[node, nb] < D[nb]
                    D[nb] = dist + W[node, nb]
                    Q[nb] = D[nb]
                end
            end
        end
        return length(Q) > 0
    end

    function create_groupings()
        groups = ones(Int, nv(G))
        for i in 1:nv(G)
            if V[i] groups[i] = 3
            elseif i in keys(Q) groups[i] = 2
            end
        end
        groups
    end
    
    function plot()
        # color nodes based on discover/visited status
        groups = create_groupings()
        node_bg_colors = [colorant"LightGrey", colorant"Grey", colorant"Black"]
        node_label_colors = [colorant"Black", colorant"Black", colorant"White"]
        nodefillc = node_bg_colors[groups]
        nodelabelc = node_label_colors[groups]
        
        # make sure the graph is plotted with the same values every time
        locs_x = Vector{Float32}([0, 1, 1, 2, 2, 3])
        locs_y = Vector{Float32}([1, 0, 2, 0, 2, 1])
        
        # use display to show multiple plots
        display(gplot(G, locs_x, locs_y, 
                nodelabel=D, edgelabel=[t[3]  for t in E], 
                      nodefillc=nodefillc, nodelabelc=nodelabelc))

    end
    
    function run_steps(n)
        for _ in 1:n
            single_step()
        end
        plot()
    end

    return run_steps
end

run_steps = dijkstra_ssp(G, E, C)
run_steps(0)
run_steps(1)
run_steps(3)
run_steps(1)
gplot(G, nodelabel=1:nv(G), edgelabel=[t[3]  for t in E])
display(gplot(G, nodelabel=1:nv(G), edgelabel=1:ne(G)))

In [ ]:
pq = PriorityQueue{Int, Int}()
pq[1]=3
pq[2] = 4
1 in keys(pq)

In [ ]:
G = DiGraph(6)
E = [
    (1, 2, 16),
    (1, 3, 13),
    (3, 2, 4),
    (2, 4, 12),
    (4, 3, 9),
    (3, 5, 14),
    (4, 6, 20),
    (5, 4, 7),
    (5, 6, 4)
]

sort!(E)

C = zeros(Int, nv(G), nv(G))

for e in E
    u, v, c = e
    add_edge!(G, u, v)
    C[u, v] = c
end

function max_flow(G, E, C, source, sink)
    F = zeros(C)
    #find augmenting path
    function find_aug_path()
        Q = Queue(Int)
        V = falses(nv(G))
        P = Dict{Int, Int}()
        
        function get_path()
            n = sink
            res = []
            while n != 0
                push!(res, n)
                n = P[n]
            end
            reverse!(res)
        end
        
        enqueue!(Q, 1)
        P[1] = 0
        
        while length(Q) > 0
            n = dequeue!(Q)
            if n == sink
                return get_path()
            end
            for nb in all_neighbors(G, n)
                if !V[nb] && F[n, nb] < C[n, nb]
                    enqueue!(Q, nb)
                    V[nb] = true
                    P[nb] = n
                end
            end
        end
        

        []
    end
    
    path = find_aug_path()
    while length(path) > 0
        f = typemax(Int)
        for i in 1:length(path)-1
            u, v = path[i], path[i+1]
            f = min(f, C[u, v] - F[u, v])
        end
        for i in 1:length(path)-1
            u, v = path[i], path[i+1]
            F[u, v] += f
        end
        path = find_aug_path()
    end
    
    F
end

F = max_flow(G, E, C, 1, 6)

locs_x = Vector{Float32}([0, 1, 1, 2, 2, 3])
locs_y = Vector{Float32}([1, 0, 2, 0, 2, 1])

# use display to show multiple plots
display(gplot(G, locs_x, locs_y, nodelabel=1:nv(G), edgelabel=["$(F[t[1], t[2]]) / $(C[t[1], t[2]])"  for t in E]))

function find_min_cut(G, E, C, F, source, sink)
    groups = ones(Int, nv(G))
    Q = Queue(Int)
    enqueue!(Q, 1)
    while length(Q) > 0
        n = dequeue!(Q)
        groups[n] = 2
        for nb in out_neighbors(G, n)
            if F[n, nb] < C[n, nb]
                enqueue!(Q, nb)
            end
        end
    end
    groups
end

grouping = find_min_cut(G, E, C, F, 1, 6)
min_cut_colors = [colorant"Blue", colorant"Red"]
nodefillc = min_cut_colors[grouping]
display(gplot(G, locs_x, locs_y, nodelabel=1:nv(G), edgelabel=["$(F[t[1], t[2]]) / $(C[t[1], t[2]])"  for t in E], nodefillc=nodefillc))

display(gplot(G, locs_x, locs_y, nodelabel=1:nv(G), edgelabel=1:ne(G)))